<a href="https://colab.research.google.com/github/martians-sheep/pl_task_recomended_csd/blob/main/neary_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Faissパッケージのインストール
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.5 MB/s eta 0:00:00


In [7]:
# ベクトルDBを利用する
import faiss

In [10]:
# Embeddingファイルを読み込み Google Driveから
# Google Driveをマウントする
from google.colab import drive
drive.mount('/content/drive')

# ファイル
embedding_file =  '/content/drive/MyDrive/PL自動割り当てEmbeddingファイル/embeddings_all.npy'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import numpy as np

# ファイルからEmbeddingを読み込む
target_embeds = np.load(embedding_file)


In [13]:
# インデックス数
d = target_embeds.shape[1]
print(d)

# Faissのインデックス生成
index = faiss.IndexFlatL2(d)

# 対象テキストをインデックスに追加
index.add(target_embeds)

1536


In [15]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00


In [16]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [17]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

In [23]:
# テキストを読み込む
txt_file_path = "./株主総会招集手続の検討報告書.txt"

with open(txt_file_path, "r", encoding="utf-8") as file:
    in_text = file.read()

In [24]:
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# numpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [25]:
# 近傍探索の実行
D, I = index.search(in_embeds, 3)

In [26]:
# 確認
print(D)
print(I)

[[0.07975949 0.13678804 0.13823856]]
[[ 0 27  1]]
